### Master Thesis Project: Finding optimal EV charging infrastructure on DFDS routes
_________________________________________________________________________________________

#### Students: Jan Gaydoul & Emanuela Zucchetto

#### Code Part 2: Using Graph Theory and the NetworkX package to find ideal locations for DFDS eTrucks charging stations

In [1]:
# Importing needed libraries

import pandas as pd
pd.set_option('display.max_columns', None)

import networkx as nx
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt


In [ ]:
# loading final pre-processed dataset
df_final = pd.read_csv('df_final.csv')


In [ ]:
### 1) Building the Graph

In [ ]:
# Create an empty graph
G = nx.Graph()

# Add nodes to the graph
for index, row in df_final.iterrows():
    from_coords = (row['FromLatitude'], row['FromLongitude'])
    to_coords = (row['ToLatitude'], row['ToLongitude'])
    G.add_node(from_coords)
    G.add_node(to_coords)

# Add edges to the graph
for index, row in df_final.iterrows():
    source = (row['FromLatitude'], row['FromLongitude'])
    target = (row['ToLatitude'], row['ToLongitude'])
    distance = row['route_distance']
    frequency = row['route_frequency']
    G.add_edge(source, target, distance=distance, frequency=frequency)

# Get the geographical boundaries of the data
min_lat, max_lat = df_final[['FromLatitude', 'ToLatitude']].min().min(), df_final[['FromLatitude', 'ToLatitude']].max().max()
min_lon, max_lon = df_final[['FromLongitude', 'ToLongitude']].min().min(), df_final[['FromLongitude', 'ToLongitude']].max().max()

# Create a Basemap object centered on the data
m = Basemap(projection='merc', llcrnrlat=min_lat-1, urcrnrlat=max_lat+1,
            llcrnrlon=min_lon-1, urcrnrlon=max_lon+1, lat_ts=0, resolution='h')

# Draw the map
m.drawcoastlines()
m.fillcontinents(color='tan', lake_color='lightblue')
m.drawcountries()

# Draw the graph
pos = {node: m(node[1], node[0]) for node in G.nodes()}
nx.draw(G, pos, with_labels=False, node_size=10)

# Show the plot
plt.show()


In [ ]:
### 2) Visualizations

In [ ]:
## Color coding the graph based on route_frequency 

# define a colormap that maps frequency values to colors
cmap = plt.cm.get_cmap('Blues')

# create a dictionary mapping edge keys to frequencies
edge_freq = dict(zip(zip(df_final['FromLatitude'], df_final['FromLongitude'], df_final['ToLatitude'], df_final['ToLongitude']), df_final['route_frequency']))

# draw the graph with colored edges
nx.draw_networkx_nodes(G, pos)
nx.draw_networkx_edges(G, pos, edge_color=[cmap(edge_freq[e]) for e in G.edges()])
plt.show()

In [ ]:
### 3) Finding the ideal locations for charging stations 